In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model


resnet_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Set Trainable Layer
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

classifier = tf.keras.Sequential()
#classifier.add(tf.keras.layers.UpSampling2D((2,2)))
#classifier.add(tf.keras.layers.UpSampling2D((2,2)))
#classifier.add(tf.keras.layers.UpSampling2D((2,2)))
classifier.add(resnet_model)
classifier.add(tf.keras.layers.GlobalAveragePooling2D(input_shape=(2,2)))
classifier.add(tf.keras.layers.Flatten())
classifier.add(tf.keras.layers.Dense(128, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.3))
classifier.add(tf.keras.layers.BatchNormalization())
classifier.add(tf.keras.layers.Dense(64, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.3))
classifier.add(tf.keras.layers.BatchNormalization())
classifier.add(tf.keras.layers.Dense(5, activation='softmax'))

classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])




# Part 2 - Fitting the CNN to the images

# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Object Classification/Training Set',
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')

#training_set = train_datagen.flow_from_directory('C:/Users/Sumit/Desktop/Object Classification/Training Set',
#                                                 target_size = (224, 224),
#                                                 batch_size = 8,
#                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Object Classification/Test Set',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

#test_set = test_datagen.flow_from_directory('C:/Users/Sumit/Desktop/Object Classification/Test Set',
#                                            target_size = (224, 224),
#                                            batch_size = 8,
 #                                           class_mode = 'categorical')

from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/Object Classification/ResNet50_Experiment1.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]


start = datetime.now()

classifier.summary()

model = classifier.fit_generator(training_set,
                         steps_per_epoch = 50,
                         epochs = 30,
                         validation_data = test_set,    
                         validation_steps = 10,
                         callbacks=callbacks ,verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

classifier.save("/content/drive/My Drive/Colab Notebooks/Object Classification/ResNet50_Experiment1.h5")
#classifier.save("C:/Users/Sumit/Desktop/Object Classification/ResNet50_Experiment1.h5")
print("Saved model to disk")

Found 426 images belonging to 5 classes.
Found 125 images belonging to 5 classes.
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               262272    
_________________________________________________________________
dropout_28 (Dropout)         (None, 128)               0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 128)               512       
_____________________________________

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1/30
50/50 [==============================] - ETA: 0s - loss: 1.3460 - accuracy: 0.5102
Epoch 00001: val_loss improved from inf to 0.55803, saving model to /content/drive/My Drive/Colab Notebooks/Object Classification/ResNet50_Experiment1.h5
50/50 [==============================] - 186s 4s/step - loss: 1.3460 - accuracy: 0.5102 - val_loss: 0.5580 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/30
50/50 [==============================] - ETA: 0s - loss: 0.7628 - accuracy: 0.7284
Epoch 00002: val_loss improved from 0.55803 to 0.29979, saving model to /content/drive/My Drive/Colab Notebooks/Object Classification/ResNet50_Experiment1.h5
50/50 [==============================] - 186s 4s/step - loss: 0.7628 - accuracy: 0.7284 - val_loss: 0.2998 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 3/30
50/50 [==============================] - ETA: 0s - loss: 0.6675 - accuracy: 0.7284
Epoch 00003: val_loss improved from 0.29979 to 0.20071, saving model to /content/drive/My Drive/Colab Notebooks/Object C

In [ ]:
+
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

conv4_block1_out (Activation)   (None, 14, 14, 1024) 0           conv4_block1_add[0][0]           
__________________________________________________________________________________________________
conv4_block2_1_conv (Conv2D)    (None, 14, 14, 256)  262400      conv4_block1_out[0][0]           
__________________________________________________________________________________________________
conv4_block2_1_bn (BatchNormali (None, 14, 14, 256)  1024        conv4_block2_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block2_1_relu (Activation (None, 14, 14, 256)  0           conv4_block2_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block2_2_conv (Conv2D)    (None, 14, 14, 256)  590080      conv4_block2_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_bloc

Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
__________________________________________________________________________________________________


In [ ]:
from tensorflow.keras import models
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('/content/drive/My Drive/Colab Notebooks/Object Classification/Validation Set/Mouse/images (1).jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = preprocess_input(test_image)
model = models.load_model('/content/drive/My Drive/Colab Notebooks/Object Classification/ResNet50_Experiment1.h5')
result = model.predict(test_image)
class_idx = np.argmax(result[0])
print(result[0],class_idx)
print(result[0][4])
#training_set.class_indices
if result[0][0] == 1:
    prediction = 'Headphone'
    print(prediction)
elif result[0][1] == 1:
    prediction = 'Keyboard'
    print(prediction)
elif result[0][2] == 1:
    prediction = 'Laptop'
    print(prediction)
elif result[0][3] == 1:
    prediction = 'Mobile'
    print(prediction)
else:
    prediction = 'Mouse'
    print(prediction)

NameError: ignored

Test Whether GPU is working?

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import tensorflow as tf
tf.test.tpu_device_name()

AttributeError: ignored

Which GPU we are using?

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4525774508651259468, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4646176974541834230
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13842703055481781806
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3732169469368498949
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

RAM Information?

In [ ]:
!cat /proc/meminfo

MemTotal:       13333556 kB
MemFree:         9142956 kB
MemAvailable:   11862616 kB
Buffers:           79480 kB
Cached:          2750336 kB
SwapCached:            0 kB
Active:          1216292 kB
Inactive:        2553924 kB
Active(anon):     866516 kB
Inactive(anon):     8528 kB
Active(file):     349776 kB
Inactive(file):  2545396 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               672 kB
Writeback:             0 kB
AnonPages:        940400 kB
Mapped:           687748 kB
Shmem:              9140 kB
Slab:             177408 kB
SReclaimable:     130756 kB
SUnreclaim:        46652 kB
KernelStack:        4448 kB
PageTables:         9256 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    3517404 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              920 kB
AnonHugePages:   

CPU Info?

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management

In [ ]:
!nvidia smi

/bin/bash: nvidia: command not found
